In [1]:
import pandas as pd
import numpy as np
from epiweeks import Week
from numpy import log, exp
from scipy.stats import lognorm
from scipy.optimize import minimize
from matplotlib import use, get_backend
import aux_functions


#use('TkAgg', force=True)
from matplotlib import pyplot as plt
#print("Switched to:", get_backend())

Load preds: 

In [2]:
df_preds= pd.read_csv('results/forecast_quantiles_cases_v3.csv')

#df_preds = df_preds.loc[df_preds.model_id != 141]
df_preds.forecast_date = pd.to_datetime(df_preds.forecast_date)
df_preds.target_end_date = pd.to_datetime(df_preds.target_end_date)


#df_preds = df_preds.loc[df_preds.date <= Week(2025, 40).startdate().strftime('%Y-%m-%d')]
            
df_preds.head()

,model,forecast_date,target_end_date,horizon,lower_95,lower_80,lower_50,pred,upper_50,upper_80,upper_95,value
0,BPagano-RtDriven,2020-10-18,2020-10-24,1,252157.46981,309027.43446,360710.91108,418324.95041,475938.98974,527622.46637,584492.43102,485474.0
1,BPagano-RtDriven,2020-10-18,2020-10-31,2,250557.34760,325076.22060,392799.04150,468292.90380,543786.76609,611509.58699,686028.46000,572162.0
2,BPagano-RtDriven,2020-10-18,2020-11-07,3,236909.92449,332846.68480,420034.10156,517226.07830,614418.05504,701605.47179,797542.23210,777428.0
3,BPagano-RtDriven,2020-10-18,2020-11-14,4,213876.19851,330803.52595,437067.18958,555524.36963,673981.54967,780245.21330,897172.54074,1028914.0
4,BPagano-RtDriven,2020-10-25,2020-10-31,1,299044.13571,364922.81525,424793.42033,491534.04269,558274.66504,618145.27012,684023.94966,572162.0


In [3]:
len(df_preds)

6867

In [6]:
df_preds = df_preds.dropna()
df_preds = df_preds.reset_index(drop=True)

In [7]:
len(df_preds)

6791

In [10]:
6867-6791

76

### Estimate the log normal parameters from the CDF: 

Fixed conditions for the entire method:

In [11]:
l_pos = tuple([col for col in df_preds.columns if 'lower' in col])
u_pos = tuple([col for col in df_preds.columns if 'upper' in col])
the_pos = list(l_pos + u_pos)
l_pos = list(l_pos)
u_pos = list(u_pos)

lvls = (0.5, 0.8, 0.95)
p = [aux_functions.quantile_pair(lvl) for lvl in lvls]
p = [i for quantile in p for i in quantile]
p.append(0.5)
ps = p.copy()
ps.sort()


In [12]:
ps

[0.025000000000000022, 0.09999999999999998, 0.25, 0.5, 0.75, 0.9, 0.975]

In [13]:
the_pos

['lower_95', 'lower_80', 'lower_50', 'upper_50', 'upper_80', 'upper_95']

In [14]:
def process_row(row):
    '''
    Estimate the lognormal parameters for each row in the dataframe
    '''
    #the_pos =  [ 'upper_50', 'upper_80', 'upper_95']

    #ps = [0.5, 0.8, 0.95]
    # pegar valores da linha
    xi_values = list(row[the_pos].values) if hasattr(row[the_pos], "values") else [row[the_pos]]
    xi_values.append(row["pred"])
    
    # ordenar
    xi_sorted = sorted(xi_values)

    out = pd.DataFrame({
    'p': ps,
    'xi': xi_sorted
    })

    out = out.loc[out.xi > 0]
    # ajustar CDF (aqui você precisa garantir que 'out' está definido de fora
    cdf_fit = aux_functions.fit_ln_CDF(
        x=out["xi"],
        Fhat=out["p"],
        weighting=0
    )
    
    return cdf_fit

def process_row2(row):
    #the_pos =  [ 'upper_50', 'upper_80', 'upper_95']

    #ps = [0.5, 0.8, 0.95]
    # pegar valores da linha
    xi_values = list(row[the_pos].values) if hasattr(row[the_pos], "values") else [row[the_pos]]
    xi_values.append(row["pred"])
    
    # ordenar
    xi_sorted = sorted(xi_values)

    out = pd.DataFrame({
    'p': ps,
    'xi': xi_sorted
    })

    # ajustar CDF (aqui você precisa garantir que 'out' está definido de fora
    cdf_fit = fit_ln_CDF(
        x=out["xi"],
        Fhat=out["p"],
        weighting=1
    )
    
    return cdf_fit

In [15]:
%%time

df_preds[['mu', 'sigma']] = df_preds.apply(process_row, axis=1, result_type = 'expand')

df_preds.head()

CPU times: total: 51min 55s
Wall time: 56min 47s


,model,forecast_date,target_end_date,horizon,lower_95,lower_80,lower_50,pred,upper_50,upper_80,upper_95,value,mu,sigma
0,BPagano-RtDriven,2020-10-18,2020-10-24,1,252157.46981,309027.43446,360710.91108,418324.95041,475938.98974,527622.46637,584492.43102,485474.0,12.934439,0.205498
1,BPagano-RtDriven,2020-10-18,2020-10-31,2,250557.34760,325076.22060,392799.04150,468292.90380,543786.76609,611509.58699,686028.46000,572162.0,13.043683,0.241115
2,BPagano-RtDriven,2020-10-18,2020-11-07,3,236909.92449,332846.68480,420034.10156,517226.07830,614418.05504,701605.47179,797542.23210,777428.0,13.138261,0.281946
3,BPagano-RtDriven,2020-10-18,2020-11-14,4,213876.19851,330803.52595,437067.18958,555524.36963,673981.54967,780245.21330,897172.54074,1028914.0,13.204400,0.321069
4,BPagano-RtDriven,2020-10-25,2020-10-31,1,299044.13571,364922.81525,424793.42033,491534.04269,558274.66504,618145.27012,684023.94966,572162.0,13.099669,0.940347


In [16]:
df_preds.isnull().sum()

model               0
forecast_date       0
target_end_date     0
horizon             0
lower_95            0
lower_80            0
lower_50            0
pred                0
upper_50            0
upper_80            0
upper_95            0
value               0
mu                 14
sigma               0
dtype: int64

In [17]:
col = 'sigma'
pos_inf_count = np.isposinf(df_preds[col]).sum()

# count only negative inf
neg_inf_count = np.isneginf(df_preds[col]).sum()

# count both +inf and -inf
total_inf_count = np.isinf(df_preds[col]).sum()

print("positive inf:", pos_inf_count)
print("negative inf:", neg_inf_count)
print("total inf:", total_inf_count)

positive inf: 0
negative inf: 0
total inf: 0


In [19]:
df_preds.to_csv('results/preds_CDF_cases_weights_equal_v3.csv', index = False)